Executive Markdown
================
In this notebook I want to demonstrate a technique for detecting stems using a corpus of words from a target language.

## Finding Suffixes
The idea: find the prefixes/suffixes

1. I have a vocabulary of words $vocab := \{w_i \}$. Let $|vocab|$ denote the size of the vocabulary.
2. I have the alphabet $\{a_j\}$ used to construct the words.

I have a putative suffix $(a_1,a_2,\ldots, a_r)$. I would like to know if it's a suffix. How do I determine it's a suffix?

1. Make a table of counts of the $r$ long *observed* tail sequences $\{ (b_1,\ldots,b_r): C_{(b_1,\ldots,b_r)}\}$ in the vocabulary of words. 
2. Let $T$ denote the number the distinct $r$ long observed sequences. Compute the probability of a sequence $(b_1,\ldots,b_r)$ occurring $C_{(b_1,\ldots,b_r)}$ times or more in the vocabulary space under the null hypothesis that each such possible sequence has probability $1/T$ of occurring:
$$ p\text{ value} := \sum_{t= C_{(b_1,\ldots,b_r)}}^{|vocab|}\binom{|vocab|}{t} p^t(1-p)^{|vocab| - t},\quad p:= 1/T $$

Alternatively, make a frequency count of the letters in the vocabulary of words, and define a probability distribution on the space of letters, $(p_{a_j})$. Then define the probability of observed sequence $(b_1,\ldots,b_r)$ occurring to be:

$$ Pr((b_1,\ldots,b_r)) = \frac{\prod_{i=1}^r p_{b_i}}{\sum_{\text{observed sequences }(d_1,\ldots,d_r)}\prod_{i=1}^r p_{d_i}}$$
Then we compute the $p$ value: 
$$p\text{ value} := \sum_{t= C_{(b_1,\ldots,b_r)}}^{|vocab|}\binom{|vocab|}{t}Pr((b_1,\ldots,b_r))^t(1-Pr((b_1,\ldots,b_r)))^{|vocab| - t}$$

If $p\text{ value} < 0.01/T$ it's a suffix or a subset of one. 


## Stems
Once I have a collection of putative suffixes $\{ sf_i \}$ I would like to know what are putative stems $\{st_n\}$. Here are my assumptions:

1. Let $V$ denote the number of words in the vocabulary $\{ word_v\}$.
1. All stems are at least 4 characters long.
2. Let $N$ be the number of all possible putative stems.
3. Let $M$ be the number of words that contain a putative suffix.

For each putative stem $st_n$ let $C_n$ denote the size of the set $\{ word_k | word_k = st_n + sf_i \text{ for some } i\}$.

* How can I determine that a putative stem $st_n$ is an actual stem?
* Do stems have a preferred list of suffixes to use to form words?

### Determining Confidence of Stems
We can model the putative stems as bins and words with putative suffixes as balls. We can model $C_n$ as a Poisson distribution $X \sim Poisson(\lambda)$ with parameter $\lambda = M/N$.

Since there are $N$ putative stems, we're performing $N$ tests and therefore we can use a hypothesis test  with $p$ value cutoff $p = 0.01/N$.

We can test $Pr(X >= C_n) < p$ to decide if putative stem $st_n$ is a stem.



### Determining Stem Groups
I want to determine if suffix $sf_i \sim sf_j$ in the sense that if $st_n$ is a stem and $st_n + sf_i$ is a word then so is (probably) $st_n + sf_j$. So how can I convince myself that $sf_i \sim sf_j$? Co-occurence. If there exists a stem $st_n$ such that both $st_n + sf_i$ and $st_n + sf_j$ are words then we say $sf_i, sf_j$ have co-occurred.

* Let $S$ denote the number of suffixes.
* Let $M_i$ be the number of occurrences of suffix $sf_i$ in my vocabulary.
* Let $N$ denote the number of stems.
* Let $N_{ij}$ denote the number of co-occurences of suffix $sf_i$ and $sf_j$.

What is the probability of $X=N_{ij}$? For each stem, the suffix $sf_i$ can only occur at most once. So the $M_i$ occurrences have to occur at $M_i$ slots. Ditto for $sf_j$. So
the probability of $X=N_{ij}$ occurring is 

\begin{align}
Pr(X=N_{ij}) &= \frac{\binom{M_i}{N_{ij}}\binom{N-M_i}{M_j -N_{ij}}}{\binom{N}{M_j}}
\end{align}


So to decide  $sf_i \sim sf_j$ we will perform $\binom{S}{2}$ pair wise tests and
decide $sf_i \sim sf_j$ if $Pr(X>=N_{ij}) < 0.01/\binom{S}{2}$ 

In [ ]:
%pylab inline

In [ ]:
from collections import Counter
from collections import defaultdict
import math
from tqdm.notebook import tqdm

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import string

nltk.download('punkt')

try:
    from bs4 import BeautifulSoup
except ModuleNotFoundError:
    # We don't have BeautifulSoup installed. We also need the lxml module
    !pip install bs4 lxml
    from bs4 import BeautifulSoup

try:
    import networkx as nx
except:
    # We don't have networkx installed. We need networkx
    !pip install networkx
    import networkx as nx
    
try:
    import graphviz
except:
    # We don't have graphviz installed. Let's install it.
    !pip install graphviz
    import graphviz

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import scipy.stats

In [ ]:
import pm_stemmer

In [ ]:
# download Mody Dick as html and then extract the content
!wget -c "https://www.gutenberg.org/files/2701/2701-h/2701-h.htm"
htmltxt= open('2701-h.htm','r').read()
soup = BeautifulSoup(htmltxt,'lxml')
content = soup.text.lower()

In [ ]:
content ="".join([x if x in string.ascii_letters else ' ' for x in content])
words = word_tokenize(content)
print("Length of word sequence: ",len(words))

vocabulary = set(words)
print("Size of vocabulary: ", len(vocabulary))

In [ ]:
def my_lgamma(x):
    """Computes log(x!)
    
    because math.lgamma(x) == log((x-1)!)"""
    return math.lgamma(x+1)

def pr_cooccurrence(n_ij, m_i,m_j, N):
    """Computes Pr(n_ij | m_i, m_j, N)"""
    numerator = my_lgamma(m_i) - my_lgamma(n_ij) - my_lgamma(m_i-n_ij) + \
    my_lgamma(N- m_i) - my_lgamma(m_j - n_ij) - my_lgamma(N -m_i - (m_j - n_ij))
    denominator = my_lgamma(N) - my_lgamma(m_j) - my_lgamma(N-m_j)
    logratio = numerator - denominator
    result = np.exp(logratio)
    return result

def compute_p_value(n_ij, m_i,m_j, N):
    """Computes Pr(X>=n_ij)"""
    
    m_j_temp, m_i_temp = min(m_i,m_j), max(m_i,m_j)
    probs = sum([pr_cooccurrence(c,m_i_temp,m_j_temp,N) \
                     for c in range(n_ij, m_j_temp+1)])
        
    return probs

In [ ]:
def compute_stem_suffix_counts(vocabulary, suffixes):
    """Computes m_i, N, stem dictionaries"""
    
    vocab = [(len(word),word) for word in vocabulary]
    vocab.sort()

    stem_dictionary = defaultdict(list)
    m_i = defaultdict(int)
    
    for l,aword in tqdm(vocab,desc='constructing stem dictionary'):
        if l < 4: # if a word is not at least 4 letters long, it's not a stem for me.
            continue
            
        for t in range(4,l+1):
            stem = aword[:t]
            suffix = aword[t:]
            if suffix in suffixes:
                stem_dictionary[stem].append(suffix)
                m_i[suffix] += 1
    
    return m_i, stem_dictionary

def compute_co_occurrence(stem_dictionary):
    """Computes n_ij for all suffixes i, j"""
    cooccurrences = defaultdict(int)
    for astem in tqdm(stem_dictionary,desc='constructing n_ij'):
        hits = stem_dictionary[astem]
        if len(hits) > 1:
            # got a hit.
            hits.sort()
            for i,s_i in enumerate(hits[:-1]):
                for j,s_j in enumerate(hits[i+1:]):
                    cooccurrences[(s_i,s_j)] +=1
                    
    return cooccurrences    

In [ ]:
def compute_num_stems(vocabulary):
    """Computes all possible stems whether valid or not. """
    vocab = [(len(word),word) for word in vocabulary]
    vocab.sort()

    stems = set()
    
    for l,aword in tqdm(vocab,desc='constructing stem dictionary'):
        if l < 4: # if a word is not at least 4 letters long, it's not a stem for me.
            continue
        stems = stems.union(set([aword[:t] for t in range(4,l+1)]))

    return len(stems)

In [ ]:
def determine_stems(stem_dictionary,num_stems, cutoff=0.01):
    """Takes the stem dictionary (stem, [suffixes])
    and decides which putative stems are actually stems.
    returns a list of stems that make the cutoff:
    
    Pr(X >= C_n) < p = cutoff/N """
    
    M = sum([len(x) for x in stem_dictionary.values()])
    N = num_stems
    lam = M/N
    p = cutoff/len(stem_dictionary) # The # of bins I'm actually testing.
    C_n = {x:len(stem_dictionary[x]) for x in stem_dictionary}
    print(M,N,lam,p)
    model = scipy.stats.poisson(mu=lam)
    scores = {x:model.sf(C_n[x]) for x in tqdm(C_n,desc='Computing Prob')}
    survivors = [x for x in scores if scores[x]< p]
    survivors.sort()
    return survivors

In [ ]:
vocabulary2 = list(vocabulary)
vocabulary2.sort()

In [ ]:
stemmer = pm_stemmer.Stemmer()
stemmer.fit(vocabulary2,verbose=True)

In [ ]:
suffixes = []
for l in stemmer.possible_suffixes:
    suffixes += [suffix for suffix in stemmer.possible_suffixes[l]]

In [ ]:
m_i, stem_dictionary = compute_stem_suffix_counts(vocabulary, suffixes)

In [ ]:
N_stems = compute_num_stems(vocabulary2)

In [ ]:
stems = determine_stems(stem_dictionary,N_stems)

In [ ]:
print(stems, len(stem_dictionary))

In [ ]:
len(stems)

In [ ]:
cooccurrence = compute_co_occurrence(stem_dictionary)

In [ ]:
N = len(stem_dictionary)
co_occ_probs = {}
for s_i,s_j in tqdm(cooccurrence,desc='Computing Pr(X=n_ij)'):
    mi = m_i[s_i]
    mj = m_i[s_j]
    n_ij = cooccurrence[(s_i,s_j)]
    co_occ_probs[(s_i,s_j)] = compute_p_value(n_ij, mi,mj, N)

In [ ]:
probs = np.array(list(co_occ_probs.values()))
print(probs.min(),probs.mean(),probs.max())

In [ ]:
T = len(probs)
cutoff = 0.01/T

In [ ]:
mi_sim_mj = [x for x in co_occ_probs if co_occ_probs[x]< cutoff]

In [ ]:
mi_sim_mj.sort()

In [ ]:
G = nx.Graph(mi_sim_mj)

In [ ]:
len(mi_sim_mj)

In [ ]:
len(G)

In [ ]:
components = [(len(x), x) for x in nx.components.connected_components(G)]
components.sort()
components.reverse()

In [ ]:
cluster_sizes = np.argwhere(bincount([x[0] for x in components])>0).flatten()
print(cluster_sizes)


In [ ]:
print("Number of equivalence classes: ", len(components))

In [ ]:
import graphviz

In [ ]:
G_subset_1 = G.subgraph(components[1][1])

H = graphviz.Graph()
for an_edge in G_subset_1.edges():
    H.edge(*an_edge)

H.render('component1_suffixes.gv',format='png')
H

In [ ]:
G_subset_2 = G.subgraph(components[2][1])

H = graphviz.Graph()
for an_edge in G_subset_2.edges():
    H.edge(*an_edge)

H.render('component2_suffixes.gv',format='png')
H

In [ ]:
G_subset_6 = G.subgraph(components[6][1])

H = graphviz.Graph()
for an_edge in G_subset_6.edges():
    H.edge(*an_edge)

H.render('component6_suffixes.gv',format='png')
H

In [ ]:
G_subset_1.edges()

In [ ]:
G_subset_1 = G.subgraph(components[1][1])
figure(figsize=(16,16))
nx.draw_networkx(G_subset_1,node_size = 1000)

In [ ]:
G_subset = G.subgraph(components[0][1])

In [ ]:
figure(figsize=(16,16))
nx.draw_networkx(G_subset)

In [ ]:
figure(figsize=(16,16))
nx.draw(G)

In [ ]:
hist(probs,bins=linspace(probs.min(),probs.max(),100));

In [ ]:
stemmer.stem('following'), stemmer.stem('abandonedly')

In [ ]:
fred = set(stem_map.values())

In [ ]:
print("Number of stems, size of vocabulary, ratio: ", \
      len(fred),len(vocabulary2), len(vocabulary2)/len(fred))

In [ ]:
stem_counts = Counter(stem_map.values())

In [ ]:
title("Distribution of stem cluster sizes")
hist(list(stem_counts.values()),bins=np.arange(stem_counts.most_common(1)[0][1]+1));

# Distribution of word sizes
I think the distribution of words by word length is approximately Poisson distributed. I would like to confirm that.

In [ ]:
size_counts = Counter([len(word) for word in vocabulary2])
mean_size = sum([l*size_counts[l] for l in size_counts])/len(vocabulary2)
mean_size

In [ ]:
l = mean_size
M = len(vocabulary2)
bins = max(list(size_counts.keys()))+ 1
pdf = np.zeros(bins)
pdf[0] = np.exp(-l)
for t in range(1,bins):
    pdf[t] = pdf[t-1]*l/t
    
expected = pdf*M

In [ ]:
sizes = np.arange(bins-1)
counts = np.array([size_counts[l] for l in sizes])

In [ ]:
title("Frequency of word sizes vs. Poisson distribution")
plot(sizes,counts,label='observed');
plot(sizes,expected[sizes],label='Poisson distribution');
legend();